<a href="https://colab.research.google.com/github/VonRottweiler/Recidivism/blob/main/Xg_Boosting_Model.v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Kaggle Recidivism Competition Team Members: Sam Scordo, Mike Lebreck, Mac Schlageter

XGN Boosting

Final: Version 1.4 was the best performing one with a score of 0.74210

# **Version 1.2**

In [ ]:
#@title
import pandas as pd
import numpy as np
import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

test = pd.read_csv('test.csv')

data = pd.read_csv('train.csv')

from sklearn.impute import SimpleImputer
mean = SimpleImputer(strategy='mean')
mode = SimpleImputer(strategy='most_frequent')

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

col = 0
for col in range(1, len(data.columns)):
  if (data[[data.columns[col]][0]].dtypes) == object:
    mode.fit(data[[data.columns[col]]])
    data[[data.columns[col]]] = mode.transform(data[[data.columns[col]]])
    data[data.columns[col]] = le.fit_transform(data[data.columns[col]])
  elif (data[[data.columns[col]][0]].dtypes) == float:
    mean.fit(data[[data.columns[col]]])
    data[[data.columns[col]]] = mean.transform(data[[data.columns[col]]])

col = 0
for col in range(1, len(test.columns)):
  if (test[[test.columns[col]][0]].dtypes) == object:
    mode.fit(test[[test.columns[col]]])
    test[[test.columns[col]]] = mode.transform(test[[test.columns[col]]])
    test[test.columns[col]] = le.fit_transform(test[test.columns[col]])
  elif (test[[test.columns[col]][0]].dtypes) == float:
    mean.fit(test[[test.columns[col]]])
    test[[test.columns[col]]] = mean.transform(test[[test.columns[col]]])

In [ ]:
X = data.drop(["ID", "Recidivism_Arrest"], axis=1)
y = data["Recidivism_Arrest"]
test = test.drop(["ID"], axis=1)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 42, shuffle = True)

In [ ]:
from xgboost import XGBClassifier
xgb = XGBClassifier(n_estimators=200)

xgb.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=200, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [ ]:
XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=np.nan,
              monotone_constraints='()', n_estimators=200, n_jobs=2,
              num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=0, gpu_id=-1,
              grow_policy=None, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=0, max_depth=6, max_leaves=None,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=200, n_jobs=2, num_parallel_tree=1, predictor='auto',
              random_state=0, ...)

In [ ]:
prediction = xgb.predict_proba(X_test)[:,1]

In [ ]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, prediction)

0.7245957553528324

In [ ]:
from sklearn.model_selection import GridSearchCV


param_test2b = {
    'min_child_weight':[6,10],
}


gsearch2b = GridSearchCV(estimator = XGBClassifier( learning_rate=0.1, n_estimators=140, max_depth=4,
                                                   subsample=0.8, colsample_bytree=0.8,objective= 'binary:logistic', 
                                                   nthread=4, scale_pos_weight=1,seed=27), param_grid = param_test2b, 
                         scoring='roc_auc',n_jobs=4, cv=2, verbose = 2)
gsearch2b.fit(X_train,y_train)

Fitting 2 folds for each of 2 candidates, totalling 4 fits


GridSearchCV(cv=2,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=0.8,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     gpu_id=None, grow_policy=None,
                                     importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=0.1, max_bin=None,
                                     max_cat_threshold=None,
                                     max_cat_to_onehot=None,
                                     max_delta_step=None, max_depth=4,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     n_estimators=140, n_jobs=None, nthread=4,
                                     num_parallel_tree=None, predictor=None, ...),
             n_jobs=4, param_grid={'min_child_weight': [6, 10]},
             scoring='roc_auc', verbose=2)

In [ ]:
GridSearchCV(cv=2,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=0.8,
                                     enable_categorical=False, gamma=None,
                                     gpu_id=None, importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=0.1, max_delta_step=None,
                                     max_depth=4, min_child_weight=None,
                                     missing=np.nan, monotone_constraints=None,
                                     n_estimators=140, n_jobs=None, nthread=4,
                                     num_parallel_tree=None, predictor=None,
                                     random_state=None, reg_alpha=None,
                                     reg_lambda=None, scale_pos_weight=1,
                                     seed=27, subsample=0.8, tree_method=None,
                                     validate_parameters=None, verbosity=None),
             n_jobs=4, param_grid={'min_child_weight': [6, 10]},
             scoring='roc_auc', verbose=2)

GridSearchCV(cv=2,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=0.8,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     gpu_id=None, grow_policy=None,
                                     importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=0.1, max_bin=None,
                                     max_cat_threshold=None,
                                     max_cat_to_onehot=None,
                                     max_delta_step=None, max_depth=4,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     n_estimators=140, n_jobs=None, nthread=4,
                                     num_parallel_tree=None, predictor=None, ...),
             n_jobs=4, param_grid={'min_child_weight': [6, 10]},
             scoring='roc_auc', verbose=2)

In [ ]:
from sklearn.model_selection import GridSearchCV
# Create the parameter grid 
param_grid = {
    'bootstrap': [True],
    'max_features': [2, 3],
    'n_estimators': [200, 300, 1000]
}
# Create a based model
rf = RandomForestClassifier()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)

In [ ]:
xgb = XGBClassifier(learning_rate=0.1, n_estimators=140, max_depth=4,
                    subsample=0.8, colsample_bytree=0.8,objective= 'binary:logistic', 
                    nthread=4, scale_pos_weight=1,seed=27, min_child_weight=10 ) 

In [ ]:
xgb.fit(X_train,y_train)
prediction = xgb.predict_proba(X_test)[:,1]
roc_auc_score(y_test, prediction) 

0.7354506410859274

In [ ]:
prediction  = xgb.predict_proba(test)[:,1]
sample_solution = pd.read_csv('sample_solution.csv')
submission_xgb = sample_solution.copy()
submission_xgb['Recidivism_Arrest'] = prediction

In [ ]:
submission_xgb

,ID,Recidivism_Arrest
0,3,0.661416
1,4,0.259540
2,7,0.631956
3,13,0.148341
4,16,0.527673
...,...,...
9006,26748,0.100161
9007,26752,0.133369
9008,26753,0.349110
9009,26754,0.203159


In [ ]:
submission_xgb.to_csv('submission.xgb.v1.2.csv', index=False)

# **Version 1.3**

In [ ]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score

# Read in data
test = pd.read_csv('test.csv')
data = pd.read_csv('train.csv')

# Impute missing values
def impute_missing_values(df):
    mean = SimpleImputer(strategy='mean')
    mode = SimpleImputer(strategy='most_frequent')
    object_cols = df.select_dtypes(include='object').columns
    float_cols = df.select_dtypes(include='float').columns
    for col in object_cols:
        mode.fit(df[[col]])
        df[[col]] = mode.transform(df[[col]])
        df[col] = LabelEncoder().fit_transform(df[col])
    for col in float_cols:
        mean.fit(df[[col]])
        df[[col]] = mean.transform(df[[col]])
    return df

data = impute_missing_values(data)
test = impute_missing_values(test)

# Split data into X and y
X = data.drop(["ID", "Recidivism_Arrest"], axis=1)
y = data["Recidivism_Arrest"]
test = test.drop(["ID"], axis=1)

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)

# Fit XGBClassifier model and make predictions
xgb = XGBClassifier(learning_rate=0.1, n_estimators=140, max_depth=4,
                    subsample=0.8, colsample_bytree=0.8, objective='binary:logistic',
                    nthread=4, seed=27, min_child_weight=10)
xgb.fit(X_train, y_train)
prediction = xgb.predict_proba(X_test)[:,1]
roc_auc_score(y_test, prediction)

# Grid search to find best hyperparameters for XGBClassifier and RandomForestClassifier
param_test_xgb = {'min_child_weight':[6,10]}
gsearch_xgb = GridSearchCV(estimator=XGBClassifier(learning_rate=0.1, n_estimators=140, max_depth=4,
                                                   subsample=0.8, colsample_bytree=0.8, objective='binary:logistic',
                                                   nthread=4, seed=27), param_grid=param_test_xgb,
                           scoring='roc_auc', n_jobs=4, cv=2, verbose=2)
gsearch_xgb.fit(X_train, y_train)

param_grid_rf = {'bootstrap': [True], 'max_features': [2, 3], 'n_estimators': [200, 300, 1000]}
rf = RandomForestClassifier()
grid_search_rf = GridSearchCV(estimator=rf, param_grid=param_grid_rf, cv=3, n_jobs=-1, verbose=2, scoring='roc_auc')
grid_search_rf.fit(X_train, y_train)
print("XGBoost Classifier: ", gsearch_xgb.best_params_)
print("Random Forest Classifier: ", grid_search_rf.best_params_)

rf.fit(X_train, y_train)
prediction = rf.predict_proba(X_test)[:,1]
roc_auc_score(y_test, prediction)

best_xgb = XGBClassifier(learning_rate=0.1, n_estimators=140, max_depth=4, subsample=0.8,
colsample_bytree=0.8, objective='binary:logistic', nthread=4, seed=27,
min_child_weight=10)
best_rf = RandomForestClassifier(bootstrap=True, max_features=2, n_estimators=1000)

best_xgb.fit(X_train, y_train)
best_rf.fit(X_train, y_train)

xgb_pred = best_xgb.predict_proba(test)[:,1]
rf_pred = best_rf.predict_proba(test)[:,1]

ensemble_pred = (xgb_pred + rf_pred) / 2



Fitting 2 folds for each of 2 candidates, totalling 4 fits
Fitting 3 folds for each of 6 candidates, totalling 18 fits
XGBoost Classifier:  {'min_child_weight': 6}
Random Forest Classifier:  {'bootstrap': True, 'max_features': 3, 'n_estimators': 1000}


In [ ]:
sample_solution = pd.read_csv('sample_solution.csv')
submission_xgb = sample_solution.copy()
submission_xgb['Recidivism_Arrest'] = ensemble_pred
submission_xgb.to_csv('submission.xgb.v1.3.csv', index=False)
submission_xgb.to_csv('submission.xgb.v1.3.csv', index=False)

# **Version 1.4**

In [ ]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score

# Read in data
test = pd.read_csv('test.csv')
data = pd.read_csv('train.csv')

# Impute missing values
def impute_missing_values(df):
    mean = SimpleImputer(strategy='mean')
    mode = SimpleImputer(strategy='most_frequent')
    object_cols = df.select_dtypes(include='object').columns
    float_cols = df.select_dtypes(include='float').columns
    for col in object_cols:
        mode.fit(df[[col]])
        df[[col]] = mode.transform(df[[col]])
        df[col] = LabelEncoder().fit_transform(df[col])
    for col in float_cols:
        mean.fit(df[[col]])
        df[[col]] = mean.transform(df[[col]])
    return df

data = impute_missing_values(data)
test = impute_missing_values(test)

# Split data into X and y
X = data.drop(["ID", "Recidivism_Arrest"], axis=1)
y = data["Recidivism_Arrest"]
test = test.drop(["ID"], axis=1)

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)

# Fit XGBClassifier model and make predictions
xgb = XGBClassifier(learning_rate=0.1, n_estimators=140, max_depth=4,
                    subsample=0.8, colsample_bytree=0.8, objective='binary:logistic',
                    nthread=4, seed=27, min_child_weight=10)
xgb.fit(X_train, y_train)
xgb_prediction = xgb.predict_proba(X_test)[:,1]
xgb_auc = roc_auc_score(y_test, xgb_prediction)

# Fit RandomForestClassifier model and make predictions
rf = RandomForestClassifier(bootstrap=True, max_features=2, n_estimators=1000)
rf.fit(X_train, y_train)
rf_prediction = rf.predict_proba(X_test)[:,1]
rf_auc = roc_auc_score(y_test, rf_prediction)

# Fit VotingClassifier model and make predictions
voting = VotingClassifier(estimators=[('xgb', xgb), ('rf', rf)], voting='soft')
voting.fit(X_train, y_train)
voting_prediction = voting.predict_proba(X_test)[:,1]
voting_auc = roc_auc_score(y_test, voting_prediction)

# Print out AUC scores for each model
print("XGBoost Classifier AUC: ", xgb_auc)
print("Random Forest Classifier AUC: ", rf_auc)
print("Voting Classifier AUC: ", voting_auc)

# Fit VotingClassifier model and make predictions for submission
voting = VotingClassifier(estimators=[('xgb', xgb), ('rf', rf)], voting='soft')
voting.fit(X, y)
voting_prediction = voting.predict_proba(test)[:,1]

# Write predictions to file
submission = pd.read_csv('sample_solution.csv')
submission['Recidivism_Arrest'] = voting_prediction
submission.to_csv('submission.xgb.v1.4.csv', index=False)


XGBoost Classifier AUC:  0.7354506410859274
Random Forest Classifier AUC:  0.7121188751998495
Voting Classifier AUC:  0.7355064422082196


# **Version 1.5**

In [ ]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score
from imblearn.over_sampling import RandomOverSampler

# Read in data
test = pd.read_csv('test.csv')
data = pd.read_csv('train.csv')

# Impute missing values
def impute_missing_values(df):
    mean = SimpleImputer(strategy='mean')
    mode = SimpleImputer(strategy='most_frequent')
    object_cols = df.select_dtypes(include='object').columns
    float_cols = df.select_dtypes(include='float').columns
    for col in object_cols:
        mode.fit(df[[col]])
        df[[col]] = mode.transform(df[[col]])
        df[col] = LabelEncoder().fit_transform(df[col])
    for col in float_cols:
        mean.fit(df[[col]])
        df[[col]] = mean.transform(df[[col]])
    return df

data = impute_missing_values(data)
test = impute_missing_values(test)

# Split data into X and y
X = data.drop(["ID", "Recidivism_Arrest"], axis=1)
y = data["Recidivism_Arrest"]
test = test.drop(["ID"], axis=1)

# Perform feature scaling
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
test_scaled = scaler.transform(test)

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42, shuffle=True)

# Perform oversampling to address class imbalance
oversampler = RandomOverSampler(random_state=42)
X_train, y_train = oversampler.fit_resample(X_train, y_train)

# Fit XGBClassifier model and make predictions
xgb = XGBClassifier(learning_rate=0.1, n_estimators=140, max_depth=4,
                    subsample=0.8, colsample_bytree=0.8, objective='binary:logistic',
                    nthread=4, seed=27, min_child_weight=10)
xgb.fit(X_train, y_train)
xgb_prediction = xgb.predict_proba(X_test)[:,1]
xgb_auc = roc_auc_score(y_test, xgb_prediction)

# Fit RandomForestClassifier model and make predictions
rf = RandomForestClassifier(bootstrap=True, max_features=2, n_estimators=1000)
rf.fit(X_train, y_train)
rf_prediction = rf.predict_proba(X_test)[:,1]
rf_auc = roc_auc_score(y_test, rf_prediction)

# Fit VotingClassifier model and make predictions
voting = VotingClassifier(estimators=[('xgb', xgb), ('rf', rf)], voting='soft')
voting.fit(X_train, y_train)
voting_prediction = voting.predict_proba(X_test)[:,1]
voting_auc = roc_auc_score(y_test, voting_prediction)

# Print out AUC scores for each model
print("XGBoost Classifier AUC: ", xgb_auc)
print("Random Forest Classifier AUC: ", rf_auc)
print("Voting Classifier AUC: ", voting_auc)

# Fit VotingClassifier model and make predictions for submission
voting = VotingClassifier(estimators=[('xgb', xgb), ('rf', rf)], voting='soft')
voting.fit(X_scaled, y)
voting_prediction = voting.predict_proba(test_scaled)[:, 1]


XGBoost Classifier AUC:  0.7421912912630491
Random Forest Classifier AUC:  0.7090542023260916
Voting Classifier AUC:  0.7394839963635224


In [ ]:
# Write predictions to file
submission = pd.read_csv('sample_solution.csv')
submission = pd.read_csv('sample_solution.csv')
submission['Recidivism_Arrest'] = voting_prediction
submission.to_csv('submission.xgb.v1.5.csv', index=False)
